<a href="https://colab.research.google.com/github/Amirosimani/ReWOO-Gemini/blob/main/ReWOO_gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --quiet datasets
# !pip install --quiet langchain langchain_community
# !pip install --quiet langchain_google_genai langchain_google_community
# !pip install --quiet wikipedia

In [1]:
from google.colab import userdata

In [27]:
MODEL = "gemini-1.5-flash"

GENERATION_CONFIG = {
    "temperature": 0.8,
    "top_p": 0.95,
    "top_k": 20,
    "candidate_count": 1,
    "max_output_tokens": 8192,
    "stop_sequences": ["STOP!"],
}

GEMINI_API_KEY = userdata.get('GEMINI')
GOOGLE_API_KEY = userdata.get('GOOGLE-API')
CSE_ID = userdata.get("CSE-ID")

## Data

[**StrategyQA**](https://paperswithcode.com/dataset/strategyqa) is a question answering benchmark where the required reasoning steps are implicit in the question, and should be inferred using a strategy. It includes 2,780 examples, each consisting of a strategy question, its decomposition, and evidence paragraphs. Questions in StrategyQA are short, topic-diverse, and cover a wide range of strategies.



In [8]:
from datasets import load_dataset

ds = load_dataset("ChilleD/StrategyQA")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
# Access the training split
train_ds = ds["train"]

In [10]:
train_ds[0]

{'qid': '4fd64bb6ce5b78ab20b6',
 'term': 'Mixed martial arts',
 'description': 'full contact combat sport',
 'question': 'Is Mixed martial arts totally original from Roman Colosseum games?',
 'answer': False,
 'facts': 'Mixed Martial arts in the UFC takes place in an enclosed structure called The Octagon. The Roman Colosseum games were fought in enclosed arenas where combatants would fight until the last man was standing. Mixed martial arts contests are stopped when one of the combatants is incapacitated. The Roman Colosseum was performed in front of crowds that numbered in the tens of thousands. Over 56,000 people attended UFC 193.'}

# LLM

## Baseline

In [11]:
from google import genai
from google.genai.types import GenerateContentConfig

In [12]:
client = genai.Client(api_key=GEMINI_API_KEY)

In [20]:
def generate(prompt, model=MODEL, config=GENERATION_CONFIG):
    response = client.models.generate_content(
        model=model,
        contents=prompt,
        config=GenerateContentConfig(**config)
    )
    return response

In [21]:
generate(prompt=train_ds[0]['question'])

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text="No, mixed martial arts (MMA) is not totally original from Roman Colosseum games. While the Colosseum featured various forms of combat that included elements found in modern MMA, such as striking, grappling, and submissions,  there are crucial differences:\n\n* **Rules and Regulations:**  Colosseum combat was largely unregulated and brutal.  There were no weight classes, rounds, or referees to enforce rules.  The goal was often death or complete incapacitation, not a points-based victory as in modern MMA.\n\n* **Training and Specialization:** Modern MMA fighters undergo years of rigorous training in various martial arts disciplines. Colosseum fighters, while likely having some training, lacked the specialized and systematic training methods availab

## Native tool call

## ReAct Agent

In [26]:
import os
from typing import List

import tiktoken
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.utilities import GoogleSearchAPIWrapper, WikipediaAPIWrapper
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction, AgentFinish, LLMResult
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
from langchain.tools import Tool

In [29]:
# --- LLM Setup ---
llm = ChatGoogleGenerativeAI(
    model=MODEL,
    google_api_key=GEMINI_API_KEY,
    convert_system_message_to_human=True,
    generation_config=GENERATION_CONFIG,
)

In [30]:
# --- Tools Setup ---
search = GoogleSearchAPIWrapper(google_api_key=GOOGLE_API_KEY, google_cse_id=CSE_ID)
wikipedia = WikipediaAPIWrapper(top_k_results=3, doc_content_chars_max=1000)

tools = [
    Tool(
        name="Google Search",
        func=search.run,
        description="Useful for finding information on current events, comparisons, or diverse perspectives.",
    ),
    Tool(
        name="Wikipedia",
        func=wikipedia.run,
        description="Useful for getting definitions and summaries of topics from Wikipedia.",
    ),
]

# --- Prompt Setup ---
prompt = hub.pull("hwchase17/react")

# --- Agent Setup ---
agent = create_react_agent(llm, tools, prompt)

In [45]:
# --- Token Counting Callback ---
class TokenCountingCallbackHandler(BaseCallbackHandler):
    def __init__(self, model_name: str):
        self.model_name = model_name
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.encoding = tiktoken.get_encoding("cl100k_base")

    def on_llm_start(self, serialized: dict, prompts: List[str], **kwargs) -> None:
        """Collect prompt tokens when LLM starts."""
        for prompt in prompts:
            self.prompt_tokens += len(self.encoding.encode(prompt))

    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        """Collect completion tokens when LLM finishes generating."""
        if response.generations:
            for generation_list in response.generations:
                for generation in generation_list:
                    if generation.text:
                        self.completion_tokens += len(self.encoding.encode(generation.text))

    def on_agent_action(self, action: AgentAction, **kwargs) -> None:
        """Increment token count on agent action."""
        if action.log:
            self.total_tokens += len(self.encoding.encode(action.log))

    def on_agent_finish(self, finish: AgentFinish, **kwargs) -> None:
        """Increment token count on agent finish."""
        if finish.log:
            self.total_tokens += len(self.encoding.encode(finish.log))

    def on_chain_end(self, outputs, **kwargs) -> None:
        """Print the total tokens used when the chain finishes."""
        self.total_tokens += self.completion_tokens + self.prompt_tokens
        print(f"Prompt tokens: {self.prompt_tokens}")
        print(f"Completion tokens: {self.completion_tokens}")
        print(f"Total tokens used in this chain: {self.total_tokens}")

    def reset(self):
        """Reset the counters for the next chain run."""
        self.total_tokens = 0
        self.prompt_tokens = 0
        self.completion_tokens = 0

In [46]:
# --- Agent Executor Setup ---
token_callback = TokenCountingCallbackHandler(MODEL)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10,
    callbacks=[token_callback],
)

In [47]:
# --- Example Usage ---
msg = agent_executor.invoke({"input": train_ds[0]["question"]})
print(msg)



> Entering new AgentExecutor chain...


/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I need to research the history of mixed martial arts and compare it to Roman Colosseum games to determine if there's a direct lineage.  Wikipedia should be a good starting point for both.

Action: Wikipedia
Action Input: Mixed martial artsPage: Mixed martial arts
Summary: Mixed martial arts (MMA) is a full-contact fighting sport based on striking and grappling, incorporating techniques from various combat sports from around the world.
In the early 20th century, various inter-stylistic contests took place throughout Japan and the countries of East Asia. At the same time, in Brazil there was a phenomenon called vale tudo, which became known for unrestricted fights between various styles such as judo, Brazilian jiu-jitsu, catch wrestling, luta livre, Muay Thai and capoeira. An early high-profile mixed bout was Kimura vs. Gracie in 1951. In mid-20th century Hong Kong, rooftop street fighting contests between different martial arts styles gave rise to Bruce Lee's hybrid martial art

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: Is Mixed martial arts totally original from Roman Colosseum games?
Thought: I need to research the history of mixed martial arts and compare it to Roman Colosseum games to determine if there's a direct lineage.  Wikipedia should be a good starting point for both.

Action: Wikipedia
Action Input: Mixed martial artsPage: Mixed martial arts
Summary: Mixed martial arts (MMA) is a full-contact fighting sport based on striking and grappling, incorporating techniques from various combat sports from around the world.
In the early 20th century, various inter-stylistic contests took place throughout Japan and the countries of East Asia. At the same time, in Brazil there was a phenomenon called vale tudo, which became known for unrestricted fights between various styles such as judo, Brazilian jiu-jitsu, catch wrestling, luta livre, Muay Thai and capoeira. An early high-profile mixed bout was Kimura vs. Gracie in 1951. In mid-20th century Hong Kong, rooftop street fighting contests betw

...

Question: Is Mixed martial arts totally original from Roman Colosseum games?
Thought: I need to research the history of mixed martial arts and compare it to Roman Colosseum games to determine if there's a direct lineage. Wikipedia is a good place to start for both.

Action: Wikipedia
Action Input: Mixed martial artsPage: Mixed martial arts
Summary: Mixed martial arts (MMA) is a full-contact fighting sport based on striking and grappling, incorporating techniques from various combat sports from around the world.
In the early 20th century, various inter-stylistic contests took place throughout Japan and the countries of East Asia. At the same time, in Brazil there was a phenomenon called vale tudo, which became known for unrestricted fights between various styles such as judo, Brazilian jiu-jitsu, catch wrestling, luta livre, Muay Thai and capoeira. An early high-profile mixed bout was Kimura vs. Gracie in 1951. In mid-20th century Hong Kong, rooftop street fighting contests between diff

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: I need to research the history of Roman Colosseum games and compare them to the history of mixed martial arts. Wikipedia should provide sufficient information.

Action: Wikipedia
Action Input: Roman ColosseumPage: Colosseum
Summary: The Colosseum ( KOL-ə-SEE-əm; Italian: Colosseo [kolosˈsɛːo], ultimately from Ancient Greek word "kolossos" meaning a large statue or giant) is an elliptical amphitheatre in the centre of the city of Rome, Italy, just east of the Roman Forum. It is the largest ancient amphitheatre ever built, and is still the largest standing amphitheatre in the world, despite its age. Construction began under the Emperor Vespasian (r. 69–79 AD) in 72 and was completed in AD 80 under his successor and heir, Titus (r. 79–81). Further modifications were made during the reign of Domitian (r. 81–96). The three emperors who were patrons of the work are known as the Flavian dynasty, and the amphitheatre was named the Flavian Amphitheatre (Latin: Amphitheatrum Flavium; It

/usr/local/lib/python3.11/dist-packages/langchain_google_genai/chat_models.py:353: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: The Wikipedia article on Mixed Martial Arts shows its evolution from various martial arts styles throughout the 20th century, primarily in Asia and Brazil.  The Colosseum article describes gladiatorial combat in ancient Rome. While both involve fighting, there's no direct lineage.  The styles and rules are vastly different.

Thought: I now know the final answer.
Final Answer: No, mixed martial arts is not totally original from Roman Colosseum games.  While both involve combat, MMA evolved from various martial arts traditions in the 20th century, not directly from gladiatorial combat in ancient Rome.
Prompt tokens: 0
Completion tokens: 0
Total tokens used in this chain: 357

> Finished chain.
{'input': 'Is Mixed martial arts totally original from Roman Colosseum games?', 'output': 'No, mixed martial arts is not totally original from Roman Colosseum games.  While both involve combat, MMA evolved from various martial arts traditions in the 20th century, not directly from gladiato

# TODO:

[x] add baseline

[] add calc (optional)

[x] callback/count tokens for react

[] ReWOO